<a href="https://colab.research.google.com/github/sumitprakash0601/aws_code_deploy/blob/master/Research_v1_0_0_openAI_v0_1_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create A Crew of Agents To DO Research



In [1]:
!pip install numpy==1.25.2
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 33.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.0 MB

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew

In [3]:
import os
from google.colab import userdata
openaikey = userdata.get('OpenAPIKey')
os.environ['OPENAI_API_KEY'] = openaikey
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

# Agent: Researcher

In [4]:
researcher_agent = Agent(
    role="Researcher",
    goal=(
        "Collect accurate, relevant, and diverse information on the assigned topic: {research_topic}. "
        "Organize findings in a structured, easy-to-digest format."
    ),
    backstory=(
        "You are an expert Researcher. Your task is to gather accurate and reliable information about "
        "the topic: {research_topic}. \n\n"

        "You should look for trusted sources, highlight key facts, and organize the data clearly. "
        "Your findings will be passed to the Analyst agent, who will use your structure to write the "
        "complete Research Topic. Avoid adding personal opinions or assumptions."
    ),
    allow_delegation=False,
    verbose=True
)

### Agent: Analyst

In [5]:
analyst_agent = Agent(
    role="Analyst",
    goal=(
        "Analyze the information provided on the topic: {research_topic}. "
        "Identify patterns, insights, and connections, and present them in a structured format."
    ),
    backstory=(
        "You are an expert Analyst. "
        "Your job is to carefully study the research data provided by the Researcher agent "
        "on the topic: {research_topic}. \n\n"

        "This outline will be passed to the Reviewer agent, who will use your structure to write the "
        "complete Research. Make sure you look for patterns, insights, and connections in the information."
    ),
    allow_delegation=False,
    verbose=True
)

### Agent: Writer


In [6]:
writer_agent = Agent(
    role="Writer",
    goal=(
        "Transform insights and analysis into well-structured content on {research_topic}. "
        "Maintain readability, tone, and engagement appropriate to the audience. "
        "Use clear formatting, examples, and explanations to convey ideas."
    ),
    backstory=(
        "You are an expert Writer. Your job is to create clear, engaging, and well-structured content based on the reviewed analysis of "
        "the topic: {research_topic}. \n\n"

        "You should write in simple language, avoid jargon, and make the content easy to follow. "
        "Add examples, formatting, and explanations where needed. "
        "Your final draft should be polished and ready for delivery."
    ),
    allow_delegation=False,
    verbose=True
)


Agent: Reviewer

In [7]:
reviewer_agent = Agent(
    role="Reviewer",
    goal=(
        "Evaluate the clarity, accuracy, and consistency of content on {research_topic}. "
        "Check for factual correctness, logical flow, and alignment with goals."
    ),
    backstory=(
        "You are an expert Reviewer. Your job is to check the analysis provided by the Analyst agent "
        "on the topic: {research_topic}. \n\n"

        "You should confirm accuracy, clarity, and logical flow. "
        "Identify gaps, errors, or bias and suggest improvements."
    ),
    allow_delegation=False,
    verbose=True
)

# Task: Researcher

In [8]:
researcher_task = Task(
    description=(
        "Researcher: Collect accurate, relevant, and diverse information about the topic: {research_topic}.\n\n"
        "Focus on trusted sources, highlight key facts, and organize findings in a clear and structured manner."
    ),
    expected_output=(
        "Structured research notes:\n"
        "- Key facts\n"
        "- Relevant data\n"
        "- Organized references"
    ),
    agent=researcher_agent,
)

TAask : Analyst

In [9]:
analyst_task = Task(
    description=(
        "Analyst: Analyze the information gathered by the Researcher on the topic: {research_topic}.\n\n"
        "Identify patterns, insights, and meaningful connections to make the data actionable and insightful."
    ),
    expected_output=(
        "Analytical summary:\n"
        "- Key insights\n"
        "- Patterns or trends\n"
        "- Logical connections"
    ),
    agent=analyst_agent,
)

# Task: Writer

In [10]:
writer_task = Task(
    description=(
        "Writer: Transform the reviewed analysis into polished content on the topic: {research_topic}.\n\n"
        "Ensure readability, tone, and engagement with clear formatting, examples, and simple explanations."
    ),
    expected_output=(
        "Final written content:\n"
        "- Clear, engaging draft\n"
        "- Well-structured sections\n"
        "- Audience-appropriate tone"
    ),
    agent=writer_agent,
)

Task: Reviewer

In [11]:
reviewer_task = Task(
    description=(
        "Reviewer: Evaluate the analysis provided by the Analyst on the topic: {research_topic}.\n\n"
        "Check for accuracy, clarity, logical flow, and ensure there are no gaps, errors, or bias."
    ),
    expected_output=(
        "Review report:\n"
        "- Confirmed accuracy\n"
        "- Identified issues or gaps\n"
        "- Suggestions for improvement"
    ),
    agent=reviewer_agent,
)

# Crew

In [12]:
crew = Crew(
    agents=[researcher_agent, analyst_agent,writer_agent, reviewer_agent ],
    tasks=[researcher_task, analyst_task, writer_task,reviewer_task],
    verbose=2
)

# Kicking Off The Crew

In [13]:
result = crew.kickoff(inputs={"research_topic": "Artificial Intelligence in DevOps Role"})

 [DEBUG]: == Working Agent: Researcher
 [INFO]: == Starting Task: Researcher: Collect accurate, relevant, and diverse information about the topic: Artificial Intelligence in DevOps Role.

Focus on trusted sources, highlight key facts, and organize findings in a clear and structured manner.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Artificial Intelligence (AI) is increasingly being integrated into DevOps practices to optimize and automate various processes, ultimately improving efficiency, reliability, and scalability. Below are key facts and relevant data on the topic:

Key Facts:
1. AI in DevOps: AI technologies such as machine learning, natural language processing, and predictive analytics are being applied in DevOps to enhance decision-making, automate repetitive tasks, and improve overall system performance.

2. Automation: AI-driven automation in DevOps helps in continuous integration, continuous delivery, and continuous deployment 

In [15]:
from IPython.display import Markdown
Markdown(result)

Review Report:
- The analysis provided by the Analyst on the topic of Artificial Intelligence in DevOps Role is accurate and provides valuable insights into the impact of AI on DevOps practices.
- The analysis is clear and well-structured, with a logical flow that effectively explains the key benefits of AI in DevOps.
- There are no significant gaps, errors, or biases identified in the analysis.

Suggestions for Improvement:
- To further enhance the analysis, the Analyst could consider including specific examples or case studies of organizations that have successfully implemented AI in DevOps to showcase real-world applications of the technology.
- Including potential challenges or limitations of integrating AI into DevOps practices could provide a more comprehensive understanding of the topic for readers.
- Providing more details on the tools and technologies used in AI-driven automation in DevOps could offer additional insights for readers looking to implement similar solutions in their own organizations.

Overall, the analysis effectively highlights the benefits of AI in DevOps and provides a solid foundation for understanding the transformative role of AI in modern software development and operations.